# Import libraries

In [1]:
import requests as r
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression

# Class

In [ ]:
class Vacation:
    def __init__(self):
        self.model

In [4]:
def get_wind_speed(r0, r1): 
    ret = {'data': {}}
    for k,v in r0['data'].items():
        ret['data'][k] = f'{math.sqrt(float(v.split()[0])**2 + float(r1["data"][k].split()[0])**2)} {v.split()[1]}'
    return ret

In [8]:
lat, lon = 40.7128, -74.0060
resp_u = r.get(f'https://api.dclimate.net/apiv3/grid-history/era5_land_wind_u-hourly/{lat}_{lon}', headers=HEADERS)
resp_v = r.get(f'https://api.dclimate.net/apiv3/grid-history/era5_land_wind_v-hourly/{lat}_{lon}', headers=HEADERS)
resp = get_wind_speed(resp_u.json(), resp_v.json())

In [15]:
resp['data'] = {k[:-6]:v for k,v in resp['data'].items()}

In [16]:
resp

{'data': {'1989-12-31 19:00:00': '10.035171456244939 mi',
  '1989-12-31 20:00:00': '11.8347460637786 mi',
  '1989-12-31 21:00:00': '13.583508991356679 mi',
  '1989-12-31 22:00:00': '12.378830321267827 mi',
  '1989-12-31 23:00:00': '10.826494343839988 mi',
  '1990-01-01 00:00:00': '9.02791571627018 mi',
  '1990-01-01 01:00:00': '6.011534783543534 mi',
  '1990-01-01 02:00:00': '8.48727943138063 mi',
  '1990-01-01 03:00:00': '8.475597816138881 mi',
  '1990-01-01 04:00:00': '9.644334922570154 mi',
  '1990-01-01 05:00:00': '10.93056436217982 mi',
  '1990-01-01 06:00:00': '11.622105783726632 mi',
  '1990-01-01 07:00:00': '12.483507527052904 mi',
  '1990-01-01 08:00:00': '12.44171015312814 mi',
  '1990-01-01 09:00:00': '12.209021227528504 mi',
  '1990-01-01 10:00:00': '12.757677363650899 mi',
  '1990-01-01 11:00:00': '14.898415544566745 mi',
  '1990-01-01 12:00:00': '15.789664820886982 mi',
  '1990-01-01 13:00:00': '16.059660325377223 mi',
  '1990-01-01 14:00:00': '17.23573566200727 mi',
  '1

We want to run a regression for wind. We will use the average wind speed?

In [19]:
def dt_range(start, end):
    date_range = pd.date_range(start=start, end=end, freq="1H").to_pydatetime().tolist()
    return [f'{str(d).split()[0]}T{str(d).split()[1]}.000Z' for d in date_range][:-1]

def dt_range_to_keys(datetime_range):
    keys = [d.split("T")[0] +' '+ d.split("T")[1].split(".")[0] for d in datetime_range]
    return keys

def select_hours(datetime_range, start, end):
    return [d for d in datetime_range if start <= int(d.split('T')[1].split(":")[0]) < end]

def get_timeseries(resp, year_start, year_end, day_start, day_end, hour_start, hour_end):
    data = resp['data']
    #generate timeseries
    ts = []
    for yr in range(year_start, year_end):
        start = f'{yr}-{day_start}'
        end = f'{yr}-{day_end}'
        ts += dt_range_to_keys(select_hours(dt_range(start, end), hour_start, hour_end))
    return {k:v for k,v in data.items() if k.split()[0]+" "+k.split()[1].split("-")[0] in ts}

In [22]:
ts = get_timeseries(resp, 2001, 2022, "09-01", "09-14", 0, 24)

In [48]:
ts

{'2001-09-01 00:00:00': '6.666107682704581 mi',
 '2001-09-01 01:00:00': '6.55334529849808 mi',
 '2001-09-01 02:00:00': '7.228108506376211 mi',
 '2001-09-01 03:00:00': '7.114055306828547 mi',
 '2001-09-01 04:00:00': '7.473510080659163 mi',
 '2001-09-01 05:00:00': '7.684161066113635 mi',
 '2001-09-01 06:00:00': '7.564847591113258 mi',
 '2001-09-01 07:00:00': '7.440394450391071 mi',
 '2001-09-01 08:00:00': '7.98861909520414 mi',
 '2001-09-01 09:00:00': '9.060282368950714 mi',
 '2001-09-01 10:00:00': '10.644620647225315 mi',
 '2001-09-01 11:00:00': '10.456714161808918 mi',
 '2001-09-01 12:00:00': '9.877030017720312 mi',
 '2001-09-01 13:00:00': '9.43671665124731 mi',
 '2001-09-01 14:00:00': '9.2061003712135 mi',
 '2001-09-01 15:00:00': '8.865071942268962 mi',
 '2001-09-01 16:00:00': '8.653503549518925 mi',
 '2001-09-01 17:00:00': '8.366410878042759 mi',
 '2001-09-01 18:00:00': '8.118846174824268 mi',
 '2001-09-01 19:00:00': '6.788834461751691 mi',
 '2001-09-01 20:00:00': '6.176213104808644 

In [24]:
def calculate_yearly_average(timeseries):
    averages = {}
    for k, v in timeseries.items():
        if int(k.split("-")[0]) in averages:
            averages[int(k.split("-")[0])] += [float(v.split()[0])]
        else:
            averages[int(k.split("-")[0])] = [float(v.split()[0])]
    return {k:sum(v)/len(v) for k,v in averages.items()}

In [25]:
calculate_yearly_average(ts)

{2001: 6.432671355942202,
 2002: 7.0418431221205635,
 2003: 6.631683874965609,
 2004: 6.475939983656552,
 2005: 6.139865908962989,
 2006: 7.056202178409009,
 2007: 6.576851897811697,
 2008: 6.372071424637091,
 2009: 6.663762741970514,
 2010: 7.925025066722101,
 2011: 6.1833483823228095,
 2012: 6.170422218886397,
 2013: 6.623403732163531,
 2014: 6.821837950681356,
 2015: 6.113126313686931,
 2016: 7.360203911571288,
 2017: 6.853135387892999,
 2018: 6.091856888303785,
 2019: 7.04513163471199,
 2020: 5.394441743869752,
 2021: 6.817988641944909}

In [43]:
years = calculate_yearly_average(ts)
y = np.array(list(years.values()))
X = np.array(list(years.keys())).reshape(-1, 1)
reg = LinearRegression().fit(X, y)

In [44]:
reg.coef_

array([-0.0089555])

In [45]:
reg.intercept_

24.618602696501725

y = (-.008955)(2022) + 24.6186

In [53]:
reg.predict([[2022]])

array([6.51057587])

In [52]:
(-.0089555)*(2022) + 24.618602696501725

6.510581696501724

In [ ]:
# wind is avg, rain is sum, snow is sum, temp is average, high temp is max, low temp is min.